In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
base_path = "/content/drive/MyDrive/Research 2/dataset/"

In [3]:
import pandas as pd

sample_data =  pd.read_csv(base_path + "train.csv")
sample_data.head()

,Unnamed: 0,article,highlights,id
0,0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


In [4]:
sample_data.shape

(287113, 4)

In [5]:
corpus=sample_data['article']
# corpus=corpus[1]
# corpus
corpus = corpus[:11]
len(corpus)

11

In [6]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.pipeline.tagger import DEFAULT_TAGGER_MODEL

from string import punctuation
from collections import Counter

In [7]:
nlp = spacy.load('en_core_web_sm')

In [16]:
def find_topic_from_corpus(corpus, i=0):
  possible_topics = {}
  doc = nlp(corpus)
  print("-------- Doc : ", (i+1), "-------- Length of Doc : ", len(doc), "--------")
  keywords = []
  stopwords = list(STOP_WORDS)
  pos_tag = ['PROPN', 'NOUN', 'VERB']

  for token in doc:
    if (token.text in stopwords or token.text in punctuation):
      continue
    if (token.pos_ in pos_tag):
      keywords.append(token.text)
  print("Len of keywords", len(keywords))
  
  freq_word = Counter(keywords)
  freq_word.most_common(5)

  # Normalization
  max_freq = Counter(keywords).most_common(1)[0][1]
  for word in freq_word.keys():
    freq_word[word] = (freq_word[word]/max_freq)
  freq_word.most_common(5)

  topics = " ".join([word for word in  freq_word.keys() if (freq_word[word] > 0.5)])
  # possible_topics["possible_s"] = topics
  possible_topics["possible_k"] = []
  # print("Possible Topics : ",topics)

  topicdoc =  nlp(topics)
  for token in topicdoc:
    if token.pos_ == "PROPN" and token.dep_ in ["compound", "ROOT", "nsubj"]:
      print(token.text, token.pos_)
      possible_topics["possible_k"].append([token.text, token.pos_, token.dep_])
    elif token.pos_ == "VERB" and token.dep_ in ["ROOT"]:
      print(token.text, token.pos_)
      possible_topics["possible_k"].append([token.text, token.pos_, token.dep_])
    elif token.pos_ == "NOUN" and token.dep_ in ["ROOT", "amod"]:
      print(token.text, token.pos_)
      possible_topics["possible_k"].append([token.text, token.pos_, token.dep_])
    
    # if token.pos_ == "PROPN":
    #   print(token.text, token.pos_, token.dep_)
    #   possible_topics["possible_k"].append([token.text, token.pos_, token.dep_])
    # else:
    #   print(token.text, token.pos_, token.dep_)
    #   possible_topics["possible_k"].append([token.text, token.pos_, token.dep_])

    possible_topics['possible_s'] = " ".join([t[0] for t in possible_topics['possible_k']])
  i += 1
  return possible_topics

In [36]:
import en_gram_core_web_trf
grammar = spacy.load("en_gram_core_web_trf")

In [37]:
i=0
for article in corpus:
  generated_topics = find_topic_from_corpus(article, i)
  print(generated_topics)
  i+=1

  doc = grammar("START "+generated_topics['possible_s'])
  final_topic = ""
  for token in doc:
    print(token.text, token.tag_)
    if "KEEP" in token.tag_:
      final_topic += token.text + " "
    if "APPEND" in token.tag_:
      final_topic += token.text
      tag_split = token.tag_.split("_")
      if tag_split[1] not in punctuation:
        final_topic += tag_split[1] + " "
  print("########## FINAL TOPIC ***", final_topic.strip(), "***")

-------- Doc :  1 -------- Length of Doc :  555 --------
Len of keywords 188
Harry PROPN
Potter PROPN
Radcliffe PROPN
{'possible_k': [['Harry', 'PROPN', 'compound'], ['Potter', 'PROPN', 'compound'], ['Radcliffe', 'PROPN', 'ROOT']], 'possible_s': 'Harry Potter Radcliffe'}
START KEEP
Harry KEEP
Potter APPEND_:
Radcliffe APPEND_?
########## FINAL TOPIC *** START Harry PotterRadcliffe ***
-------- Doc :  2 -------- Length of Doc :  836 --------
Len of keywords 253
Leifman PROPN
says VERB
{'possible_k': [['Leifman', 'PROPN', 'nsubj'], ['says', 'VERB', 'ROOT']], 'possible_s': 'Leifman says'}
START KEEP
Leifman KEEP
says APPEND_.
########## FINAL TOPIC *** START Leifman says ***
-------- Doc :  3 -------- Length of Doc :  898 --------
Len of keywords 281
bridge PROPN
said VERB
{'possible_k': [['bridge', 'PROPN', 'nsubj'], ['said', 'VERB', 'ROOT']], 'possible_s': 'bridge said'}
START APPEND_The
bridge KEEP
said APPEND_.
########## FINAL TOPIC *** STARTThe bridge said ***
-------- Doc :  4 ----

In [38]:
article_sample  = '''
thermodynamics, science of the relationship between heat, work, temperature, and energy. In broad terms, thermodynamics deals with the transfer of energy from one place to another and from one form to another. The key concept is that heat is a form of energy corresponding to a definite amount of mechanical work.
Heat was not formally recognized as a form of energy until about 1798, when Count Rumford (Sir Benjamin Thompson), a British military engineer, noticed that limitless amounts of heat could be generated in the boring of cannon barrels and that the amount of heat generated is proportional to the work done in turning a blunt boring tool. Rumford’s observation of the proportionality between heat generated and work done lies at the foundation of thermodynamics. Another pioneer was the French military engineer Sadi Carnot, who introduced the concept of the heat-engine cycle and the principle of reversibility in 1824. Carnot’s work concerned the limitations on the maximum amount of work that can be obtained from a steam engine operating with a high-temperature heat transfer as its driving force. Later that century, these ideas were developed by Rudolf Clausius, a German mathematician and physicist, into the first and second laws of thermodynamics, respectively.
The most important laws of thermodynamics are:
The zeroth law of thermodynamics. When two systems are each in thermal equilibrium with a third system, the first two systems are in thermal equilibrium with each other. This property makes it meaningful to use thermometers as the “third system” and to define a temperature scale.
The first law of thermodynamics, or the law of conservation of energy. The change in a system’s internal energy is equal to the difference between heat added to the system from its surroundings and work done by the system on its surroundings.
The second law of thermodynamics. Heat does not flow spontaneously from a colder region to a hotter region, or, equivalently, heat at a given temperature cannot be converted entirely into work. Consequently, the entropy of a closed system, or heat energy per unit temperature, increases over time toward some maximum value. Thus, all closed systems tend toward an equilibrium state in which entropy is at a maximum and no energy is available to do useful work.
The third law of thermodynamics. The entropy of a perfect crystal of an element in its most stable form tends to zero as the temperature approaches absolute zero. This allows an absolute scale for entropy to be established that, from a statistical point of view, determines the degree of randomness or disorder in a system.
Although thermodynamics developed rapidly during the 19th century in response to the need to optimize the performance of steam engines, the sweeping generality of the laws of thermodynamics makes them applicable to all physical and biological systems. In particular, the laws of thermodynamics give a complete description of all changes in the energy state of any system and its ability to perform useful work on its surroundings.
This article covers classical thermodynamics, which does not involve the consideration of individual atoms or molecules. Such concerns are the focus of the branch of thermodynamics known as statistical thermodynamics, or statistical mechanics, which expresses macroscopic thermodynamic properties in terms of the behaviour of individual particles and their interactions. It has its roots in the latter part of the 19th century, when atomic and molecular theories of matter began to be generally accepted.
'''
generated_topics = find_topic_from_corpus(article_sample)
print(generated_topics)
print(generated_topics["possible_k"])

doc = grammar("START "+generated_topics['possible_s'])
final_topic = ""
for token in doc:
  print(token.text, token.tag_)
  if "KEEP" in token.tag_:
    final_topic += token.text + " "
  if "APPEND" in token.tag_:
    final_topic += token.text + " "
    tag_split = token.tag_.split("_")
    if tag_split[1] not in punctuation:
        final_topic += tag_split[1] + " "
print("########## FINAL TOPIC ***", final_topic.strip(), "***")

-------- Doc :  1 -------- Length of Doc :  646 --------
Len of keywords 218
thermodynamics NOUN
system NOUN
{'possible_k': [['thermodynamics', 'NOUN', 'amod'], ['system', 'NOUN', 'ROOT']], 'possible_s': 'thermodynamics system'}
[['thermodynamics', 'NOUN', 'amod'], ['system', 'NOUN', 'ROOT']]
START KEEP
thermodynamics KEEP
system REPLACE_?
########## FINAL TOPIC *** START thermodynamics ***


In [39]:
article_sample='''
While a number of definitions of artificial intelligence (AI) have surfaced over the last few decades, John McCarthy offers the following definition in this 2004 paper (PDF, 106 KB) (link resides outside IBM), " It is the science and engineering of making intelligent machines, especially intelligent computer programs. It is related to the similar task of using computers to understand human intelligence, but AI does not have to confine itself to methods that are biologically observable."
However, decades before this definition, the birth of the artificial intelligence conversation was denoted by Alan Turing's seminal work, "Computing Machinery and Intelligence" (PDF, 89.8 KB) (link resides outside of IBM), which was published in 1950. In this paper, Turing, often referred to as the "father of computer science", asks the following question, "Can machines think?"  From there, he offers a test, now famously known as the "Turing Test", where a human interrogator would try to distinguish between a computer and human text response. While this test has undergone much scrutiny since its publish, it remains an important part of the history of AI as well as an ongoing concept within philosophy as it utilizes ideas around linguistics.
Stuart Russell and Peter Norvig then proceeded to publish, Artificial Intelligence: A Modern Approach (link resides outside IBM), becoming one of the leading textbooks in the study of AI. In it, they delve into four potential goals or definitions of AI, which differentiates computer systems on the basis of rationality and thinking vs. acting:
Human approach:
Systems that think like humans
Systems that act like humans
Ideal approach:
Systems that think rationally
Systems that act rationally
Alan Turing’s definition would have fallen under the category of “systems that act like humans.”
At its simplest form, artificial intelligence is a field, which combines computer science and robust datasets, to enable problem-solving. It also encompasses sub-fields of machine learning and deep learning, which are frequently mentioned in conjunction with artificial intelligence. These disciplines are comprised of AI algorithms which seek to create expert systems which make predictions or classifications based on input data.
Today, a lot of hype still surrounds AI development, which is expected of any new emerging technology in the market. As noted in Gartner’s hype cycle (link resides outside IBM), product innovations like, self-driving cars and personal assistants, follow “a typical progression of innovation, from overenthusiasm through a period of disillusionment to an eventual understanding of the innovation’s relevance and role in a market or domain.” As Lex Fridman notes here (01:08:05) (link resides outside IBM) in his MIT lecture in 2019, we are at the peak of inflated expectations, approaching the trough of disillusionment.
As conversations emerge around the ethics of AI, we can begin to see the initial glimpses of the trough of disillusionment. To read more on where IBM stands within the conversation around AI ethics, read more here.
'''
generated_topics=find_topic_from_corpus(article_sample)
print(generated_topics)
print(generated_topics['possible_k'])

doc = grammar("START "+generated_topics['possible_s'])
final_topic = ""
for token in doc:
  print(token.text, token.tag_)
  if "KEEP" in token.tag_:
    final_topic += token.text + " "
  if "APPEND" in token.tag_:
    final_topic += token.text + " "
    tag_split = token.tag_.split("_")
    if tag_split[1] not in punctuation:
        final_topic += tag_split[1] + " "
print("########## FINAL TOPIC ***", final_topic.strip(), "***")

-------- Doc :  1 -------- Length of Doc :  589 --------
Len of keywords 224
AI PROPN
resides VERB
IBM PROPN
{'possible_k': [['AI', 'PROPN', 'compound'], ['resides', 'VERB', 'ROOT'], ['IBM', 'PROPN', 'compound']], 'possible_s': 'AI resides IBM'}
[['AI', 'PROPN', 'compound'], ['resides', 'VERB', 'ROOT'], ['IBM', 'PROPN', 'compound']]
START KEEP
AI KEEP
resides APPEND_in
IBM APPEND_.
########## FINAL TOPIC *** START AI resides in IBM ***


In [40]:
article_sample='''
What is a Network?
A network consists of two or more computers that are linked in order to share resources (such as printers and CDs), exchange files, or allow electronic communications. The computers on a network may be linked through cables, telephone lines, radio waves, satellites, or infrared light beams.
Two very common types of networks include:
Local Area Network (LAN)
Wide Area Network (WAN)
You may also see references to a Metropolitan Area Networks (MAN), a Wireless LAN (WLAN), or a Wireless WAN (WWAN).
Local Area Network
A Local Area Network (LAN) is a network that is confined to a relatively small area. It is generally limited to a geographic area such as a writing lab, school, or building.
Computers connected to a network are broadly categorized as servers or workstations. Servers are generally not used by humans directly, but rather run continuously to provide "services" to the other computers (and their human users) on the network. Services provided can include printing and faxing, software hosting, file storage and sharing, messaging, data storage and retrieval, complete access control (security) for the network's resources, and many others.
Workstations are called such because they typically do have a human user which interacts with the network through them. Workstations were traditionally considered a desktop, consisting of a computer, keyboard, display, and mouse, or a laptop, with with integrated keyboard, display, and touchpad. With the advent of the tablet computer, and the touch screen devices such as iPad and iPhone, our definition of workstation is quickly evolving to include those devices, because of their ability to interact with the network and utilize network services.
Servers tend to be more powerful than workstations, although configurations are guided by needs. For example, a group of servers might be located in a secure area, away from humans, and only accessed through the network. In such cases, it would be common for the servers to operate without a dedicated display or keyboard. However, the size and speed of the server's processor(s), hard drive, and main memory might add dramatically to the cost of the system. On the other hand, a workstation might not need as much storage or working memory, but might require an expensive display to accommodate the needs of its user. Every computer on a network should be appropriately configured for its use.
On a single LAN, computers and servers may be connected by cables or wirelessly. Wireless access to a wired network is made possible by wireless access points (WAPs). These WAP devices provide a bridge between computers and networks. A typical WAP might have the theoretical capacity to connect hundreds or even thousands of wireless users to a network, although practical capacity might be far less.
Nearly always servers will be connected by cables to the network, because the cable connections remain the fastest. Workstations which are stationary (desktops) are also usually connected by a cable to the network, although the cost of wireless adapters has dropped to the point that, when installing workstations in an existing facility with inadequate wiring, it can be easier and less expensive to use wireless for a desktop.
See the Topology, Cabling, and Hardware sections of this tutorial for more information on the configuration of a LAN.
Wide Area Network
Wide Area Networks (WANs) connect networks in larger geographic areas, such as Florida, the United States, or the world. Dedicated transoceanic cabling or satellite uplinks may be used to connect this type of global network.
Using a WAN, schools in Florida can communicate with places like Tokyo in a matter of seconds, without paying enormous phone bills. Two users a half-world apart with workstations equipped with microphones and a webcams might teleconference in real time. A WAN is complicated. It uses multiplexers, bridges, and routers to connect local and metropolitan networks to global communications networks like the Internet. To users, however, a WAN will not appear to be much different than a LAN.
'''
generated_topics=find_topic_from_corpus(article_sample)
print(generated_topics)
print(generated_topics['possible_k'])

doc = grammar("START "+generated_topics['possible_s'])
final_topic = ""
for token in doc:
  print(token.text, token.tag_)
  if "KEEP" in token.tag_:
    final_topic += token.text + " "
  if "APPEND" in token.tag_:
    final_topic += token.text + " "
    tag_split = token.tag_.split("_")
    if tag_split[1] not in punctuation:
        final_topic += tag_split[1] + " "
print("########## FINAL TOPIC ***", final_topic.strip(), "***")

-------- Doc :  1 -------- Length of Doc :  793 --------
Len of keywords 286
network NOUN
{'possible_k': [['network', 'NOUN', 'ROOT']], 'possible_s': 'network'}
[['network', 'NOUN', 'ROOT']]
START KEEP
network APPEND_?
########## FINAL TOPIC *** START network ***


Some rules for grammatically correct topic
1. PROPN - compound
2. PROPN - root
3. PROPN - nsubj
4. VERB - ROOT
5. NOUN - root
6. NOUN - amod

# **trying to frame grammatically correct sentence**

In [32]:
# !pip install spacy[transformers]==3.3.1

In [33]:
# !pip install https://huggingface.co/hsengiv/en_gram_core_web_trf/resolve/main/en_gram_core_web_trf-any-py3-none-any.whl

In [34]:
# !pip install spacy-transformers

In [31]:
# Using spacy.load().
import spacy
import spacy_transformers
import en_gram_core_web_trf
grammar = spacy.load("en_gram_core_web_trf")

doc = grammar("START "+generated_topics['possible_s'])
for i in doc:
  print(i.text, i.tag_)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:887: UserWarning: [W095] Model 'en_gram_core_web_trf' (0.0.1) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.5.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  model (Model): The parent model.


START KEEP
Harry KEEP
Potter APPEND_:
Radcliffe APPEND_?
